In [15]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from scipy.stats import entropy

df = pd.read_csv("../raw/audible.csv")
df


,Review_Text,Posted_Date,Rating,Average_Rating,Num_of_Ratings,Helpfulness,Reviewer,review_title,Link,Rating_Distribution,Rating_of_Performance,Rating_of_Story
0,I enjoyed both The Martian and Artemis so I pr...,05-04-21,5,4.9,182379,656,Davidgonzalezsr,Bazinga,https://www.audible.com/pd/Project-Hail-Mary-A...,"162833,15895,2619,672,360","156733,7989,1298,287,227","145112,16872,2818,763,416"
1,Awesome story telling Great build up of the c...,05-05-21,5,4.9,182379,159,David,Absolutely Great way better than Artemis,https://www.audible.com/pd/Project-Hail-Mary-A...,"162833,15895,2619,672,360","156733,7989,1298,287,227","145112,16872,2818,763,416"
2,Let me start off by saying that I strongly enj...,05-05-21,5,4.9,182379,157,Roswatheist,Highest Order of Geekgasm Medal,https://www.audible.com/pd/Project-Hail-Mary-A...,"162833,15895,2619,672,360","156733,7989,1298,287,227","145112,16872,2818,763,416"
3,Every once in a while Ill finish a book and ca...,05-11-21,5,4.9,182379,34,J. Kenney,"So good, its depressing",https://www.audible.com/pd/Project-Hail-Mary-A...,"162833,15895,2619,672,360","156733,7989,1298,287,227","145112,16872,2818,763,416"
4,In the Martian his high school science lecture...,05-08-21,4,4.9,182379,20,Celia,NOT the Martian,https://www.audible.com/pd/Project-Hail-Mary-A...,"162833,15895,2619,672,360","156733,7989,1298,287,227","145112,16872,2818,763,416"
...,...,...,...,...,...,...,...,...,...,...,...,...
100085,This rendition is delicious If you have never ...,02-09-09,5,4.6,55145,0,Elena,Its a classic,https://www.audible.com/pd/The-Hitchhikers-Gui...,"40868,9576,3215,929,557","40758,5054,1135,232,192","34621,7996,3097,994,594"
100086,Stephen Fry is a master Enjoyed hearing him br...,05-12-18,5,4.6,55145,0,Colleen,Love this book,https://www.audible.com/pd/The-Hitchhikers-Gui...,"40868,9576,3215,929,557","40758,5054,1135,232,192","34621,7996,3097,994,594"
100087,narrator was absolutely perfect for such a goo...,04-27-18,5,4.6,55145,0,Brandon Toppins,Sci fi so good,https://www.audible.com/pd/The-Hitchhikers-Gui...,"40868,9576,3215,929,557","40758,5054,1135,232,192","34621,7996,3097,994,594"
100088,This is the kind of book that allows your imag...,10-02-10,5,4.6,55145,0,Bill,Perfect book for a road trip,https://www.audible.com/pd/The-Hitchhikers-Gui...,"40868,9576,3215,929,557","40758,5054,1135,232,192","34621,7996,3097,994,594"


# length

In [16]:
def count_words(column):
    return column.apply(lambda x: len(str(x).split()))

# 각 열의 단어 수 계산하여 새로운 열 추가
df['title_length'] = count_words(df['review_title'])
df['text_length'] = count_words(df['Review_Text'])

# 결과 출력
df

,Review_Text,Posted_Date,Rating,Average_Rating,Num_of_Ratings,Helpfulness,Reviewer,review_title,Link,Rating_Distribution,Rating_of_Performance,Rating_of_Story,title_length,text_length
0,I enjoyed both The Martian and Artemis so I pr...,05-04-21,5,4.9,182379,656,Davidgonzalezsr,Bazinga,https://www.audible.com/pd/Project-Hail-Mary-A...,"162833,15895,2619,672,360","156733,7989,1298,287,227","145112,16872,2818,763,416",1,94
1,Awesome story telling Great build up of the c...,05-05-21,5,4.9,182379,159,David,Absolutely Great way better than Artemis,https://www.audible.com/pd/Project-Hail-Mary-A...,"162833,15895,2619,672,360","156733,7989,1298,287,227","145112,16872,2818,763,416",6,46
2,Let me start off by saying that I strongly enj...,05-05-21,5,4.9,182379,157,Roswatheist,Highest Order of Geekgasm Medal,https://www.audible.com/pd/Project-Hail-Mary-A...,"162833,15895,2619,672,360","156733,7989,1298,287,227","145112,16872,2818,763,416",5,160
3,Every once in a while Ill finish a book and ca...,05-11-21,5,4.9,182379,34,J. Kenney,"So good, its depressing",https://www.audible.com/pd/Project-Hail-Mary-A...,"162833,15895,2619,672,360","156733,7989,1298,287,227","145112,16872,2818,763,416",4,109
4,In the Martian his high school science lecture...,05-08-21,4,4.9,182379,20,Celia,NOT the Martian,https://www.audible.com/pd/Project-Hail-Mary-A...,"162833,15895,2619,672,360","156733,7989,1298,287,227","145112,16872,2818,763,416",3,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100085,This rendition is delicious If you have never ...,02-09-09,5,4.6,55145,0,Elena,Its a classic,https://www.audible.com/pd/The-Hitchhikers-Gui...,"40868,9576,3215,929,557","40758,5054,1135,232,192","34621,7996,3097,994,594",3,23
100086,Stephen Fry is a master Enjoyed hearing him br...,05-12-18,5,4.6,55145,0,Colleen,Love this book,https://www.audible.com/pd/The-Hitchhikers-Gui...,"40868,9576,3215,929,557","40758,5054,1135,232,192","34621,7996,3097,994,594",3,15
100087,narrator was absolutely perfect for such a goo...,04-27-18,5,4.6,55145,0,Brandon Toppins,Sci fi so good,https://www.audible.com/pd/The-Hitchhikers-Gui...,"40868,9576,3215,929,557","40758,5054,1135,232,192","34621,7996,3097,994,594",4,16
100088,This is the kind of book that allows your imag...,10-02-10,5,4.6,55145,0,Bill,Perfect book for a road trip,https://www.audible.com/pd/The-Hitchhikers-Gui...,"40868,9576,3215,929,557","40758,5054,1135,232,192","34621,7996,3097,994,594",6,43


# time_lapesd

In [17]:
print(df['Posted_Date'].head())  # 데이터의 일부를 출력

0    05-04-21
1    05-05-21
2    05-05-21
3    05-11-21
4    05-08-21
Name: Posted_Date, dtype: object


In [18]:
# Crawled_date 열 생성 및 값 채우기
df['Crawled_date'] = '24-12-05'

# Posted_Date의 형식을 변경 (05-04-21 -> 21-05-04로 변환)
df['Posted_Date'] = pd.to_datetime(df['Posted_Date'], format='%m-%d-%y', errors='coerce')
df['Crawled_date'] = pd.to_datetime(df['Crawled_date'], format='%y-%m-%d', errors='coerce')

# 날짜 차이 계산 (Crawled_date - Posted_Date)
df['time_lapsed'] = (df['Crawled_date'] - df['Posted_Date']).dt.days  # 일(day) 단위로 계산

# 결과 확인
df

,Review_Text,Posted_Date,Rating,Average_Rating,Num_of_Ratings,Helpfulness,Reviewer,review_title,Link,Rating_Distribution,Rating_of_Performance,Rating_of_Story,title_length,text_length,Crawled_date,time_lapsed
0,I enjoyed both The Martian and Artemis so I pr...,2021-05-04,5,4.9,182379,656,Davidgonzalezsr,Bazinga,https://www.audible.com/pd/Project-Hail-Mary-A...,"162833,15895,2619,672,360","156733,7989,1298,287,227","145112,16872,2818,763,416",1,94,2024-12-05,1311
1,Awesome story telling Great build up of the c...,2021-05-05,5,4.9,182379,159,David,Absolutely Great way better than Artemis,https://www.audible.com/pd/Project-Hail-Mary-A...,"162833,15895,2619,672,360","156733,7989,1298,287,227","145112,16872,2818,763,416",6,46,2024-12-05,1310
2,Let me start off by saying that I strongly enj...,2021-05-05,5,4.9,182379,157,Roswatheist,Highest Order of Geekgasm Medal,https://www.audible.com/pd/Project-Hail-Mary-A...,"162833,15895,2619,672,360","156733,7989,1298,287,227","145112,16872,2818,763,416",5,160,2024-12-05,1310
3,Every once in a while Ill finish a book and ca...,2021-05-11,5,4.9,182379,34,J. Kenney,"So good, its depressing",https://www.audible.com/pd/Project-Hail-Mary-A...,"162833,15895,2619,672,360","156733,7989,1298,287,227","145112,16872,2818,763,416",4,109,2024-12-05,1304
4,In the Martian his high school science lecture...,2021-05-08,4,4.9,182379,20,Celia,NOT the Martian,https://www.audible.com/pd/Project-Hail-Mary-A...,"162833,15895,2619,672,360","156733,7989,1298,287,227","145112,16872,2818,763,416",3,78,2024-12-05,1307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100085,This rendition is delicious If you have never ...,2009-02-09,5,4.6,55145,0,Elena,Its a classic,https://www.audible.com/pd/The-Hitchhikers-Gui...,"40868,9576,3215,929,557","40758,5054,1135,232,192","34621,7996,3097,994,594",3,23,2024-12-05,5778
100086,Stephen Fry is a master Enjoyed hearing him br...,2018-05-12,5,4.6,55145,0,Colleen,Love this book,https://www.audible.com/pd/The-Hitchhikers-Gui...,"40868,9576,3215,929,557","40758,5054,1135,232,192","34621,7996,3097,994,594",3,15,2024-12-05,2399
100087,narrator was absolutely perfect for such a goo...,2018-04-27,5,4.6,55145,0,Brandon Toppins,Sci fi so good,https://www.audible.com/pd/The-Hitchhikers-Gui...,"40868,9576,3215,929,557","40758,5054,1135,232,192","34621,7996,3097,994,594",4,16,2024-12-05,2414
100088,This is the kind of book that allows your imag...,2010-10-02,5,4.6,55145,0,Bill,Perfect book for a road trip,https://www.audible.com/pd/The-Hitchhikers-Gui...,"40868,9576,3215,929,557","40758,5054,1135,232,192","34621,7996,3097,994,594",6,43,2024-12-05,5178


# star_dev

In [19]:
df['Deviation of star ratings'] = abs(df['Rating'] - df['Average_Rating'])
df

,Review_Text,Posted_Date,Rating,Average_Rating,Num_of_Ratings,Helpfulness,Reviewer,review_title,Link,Rating_Distribution,Rating_of_Performance,Rating_of_Story,title_length,text_length,Crawled_date,time_lapsed,Deviation of star ratings
0,I enjoyed both The Martian and Artemis so I pr...,2021-05-04,5,4.9,182379,656,Davidgonzalezsr,Bazinga,https://www.audible.com/pd/Project-Hail-Mary-A...,"162833,15895,2619,672,360","156733,7989,1298,287,227","145112,16872,2818,763,416",1,94,2024-12-05,1311,0.1
1,Awesome story telling Great build up of the c...,2021-05-05,5,4.9,182379,159,David,Absolutely Great way better than Artemis,https://www.audible.com/pd/Project-Hail-Mary-A...,"162833,15895,2619,672,360","156733,7989,1298,287,227","145112,16872,2818,763,416",6,46,2024-12-05,1310,0.1
2,Let me start off by saying that I strongly enj...,2021-05-05,5,4.9,182379,157,Roswatheist,Highest Order of Geekgasm Medal,https://www.audible.com/pd/Project-Hail-Mary-A...,"162833,15895,2619,672,360","156733,7989,1298,287,227","145112,16872,2818,763,416",5,160,2024-12-05,1310,0.1
3,Every once in a while Ill finish a book and ca...,2021-05-11,5,4.9,182379,34,J. Kenney,"So good, its depressing",https://www.audible.com/pd/Project-Hail-Mary-A...,"162833,15895,2619,672,360","156733,7989,1298,287,227","145112,16872,2818,763,416",4,109,2024-12-05,1304,0.1
4,In the Martian his high school science lecture...,2021-05-08,4,4.9,182379,20,Celia,NOT the Martian,https://www.audible.com/pd/Project-Hail-Mary-A...,"162833,15895,2619,672,360","156733,7989,1298,287,227","145112,16872,2818,763,416",3,78,2024-12-05,1307,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100085,This rendition is delicious If you have never ...,2009-02-09,5,4.6,55145,0,Elena,Its a classic,https://www.audible.com/pd/The-Hitchhikers-Gui...,"40868,9576,3215,929,557","40758,5054,1135,232,192","34621,7996,3097,994,594",3,23,2024-12-05,5778,0.4
100086,Stephen Fry is a master Enjoyed hearing him br...,2018-05-12,5,4.6,55145,0,Colleen,Love this book,https://www.audible.com/pd/The-Hitchhikers-Gui...,"40868,9576,3215,929,557","40758,5054,1135,232,192","34621,7996,3097,994,594",3,15,2024-12-05,2399,0.4
100087,narrator was absolutely perfect for such a goo...,2018-04-27,5,4.6,55145,0,Brandon Toppins,Sci fi so good,https://www.audible.com/pd/The-Hitchhikers-Gui...,"40868,9576,3215,929,557","40758,5054,1135,232,192","34621,7996,3097,994,594",4,16,2024-12-05,2414,0.4
100088,This is the kind of book that allows your imag...,2010-10-02,5,4.6,55145,0,Bill,Perfect book for a road trip,https://www.audible.com/pd/The-Hitchhikers-Gui...,"40868,9576,3215,929,557","40758,5054,1135,232,192","34621,7996,3097,994,594",6,43,2024-12-05,5178,0.4


# readability

In [21]:
pip install textstat

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 105 kB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 14.2 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [22]:
import textstat


# 4. Readability 점수를 계산할 열 이름 설정
text_column = "Review_Text"  # CSV 파일의 텍스트가 저장된 열 이름

# 5. 새로운 열 추가 및 점수 계산
df["FOG Index"] = df[text_column].apply(lambda text: textstat.gunning_fog(str(text)))
df["Flesch Reading Ease"] = df[text_column].apply(lambda text: textstat.flesch_reading_ease(str(text)))

df

,Review_Text,Posted_Date,Rating,Average_Rating,Num_of_Ratings,Helpfulness,Reviewer,review_title,Link,Rating_Distribution,Rating_of_Performance,Rating_of_Story,title_length,text_length,Crawled_date,time_lapsed,Deviation of star ratings,FOG Index,Flesch Reading Ease
0,I enjoyed both The Martian and Artemis so I pr...,2021-05-04,5,4.9,182379,656,Davidgonzalezsr,Bazinga,https://www.audible.com/pd/Project-Hail-Mary-A...,"162833,15895,2619,672,360","156733,7989,1298,287,227","145112,16872,2818,763,416",1,94,2024-12-05,1311,0.1,42.28,-7.02
1,Awesome story telling Great build up of the c...,2021-05-05,5,4.9,182379,159,David,Absolutely Great way better than Artemis,https://www.audible.com/pd/Project-Hail-Mary-A...,"162833,15895,2619,672,360","156733,7989,1298,287,227","145112,16872,2818,763,416",6,46,2024-12-05,1310,0.1,21.01,41.71
2,Let me start off by saying that I strongly enj...,2021-05-05,5,4.9,182379,157,Roswatheist,Highest Order of Geekgasm Medal,https://www.audible.com/pd/Project-Hail-Mary-A...,"162833,15895,2619,672,360","156733,7989,1298,287,227","145112,16872,2818,763,416",5,160,2024-12-05,1310,0.1,67.00,-74.01
3,Every once in a while Ill finish a book and ca...,2021-05-11,5,4.9,182379,34,J. Kenney,"So good, its depressing",https://www.audible.com/pd/Project-Hail-Mary-A...,"162833,15895,2619,672,360","156733,7989,1298,287,227","145112,16872,2818,763,416",4,109,2024-12-05,1304,0.1,44.33,-13.79
4,In the Martian his high school science lecture...,2021-05-08,4,4.9,182379,20,Celia,NOT the Martian,https://www.audible.com/pd/Project-Hail-Mary-A...,"162833,15895,2619,672,360","156733,7989,1298,287,227","145112,16872,2818,763,416",3,78,2024-12-05,1307,0.9,34.28,0.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100085,This rendition is delicious If you have never ...,2009-02-09,5,4.6,55145,0,Elena,Its a classic,https://www.audible.com/pd/The-Hitchhikers-Gui...,"40868,9576,3215,929,557","40758,5054,1135,232,192","34621,7996,3097,994,594",3,23,2024-12-05,5778,0.4,16.16,39.67
100086,Stephen Fry is a master Enjoyed hearing him br...,2018-05-12,5,4.6,55145,0,Colleen,Love this book,https://www.audible.com/pd/The-Hitchhikers-Gui...,"40868,9576,3215,929,557","40758,5054,1135,232,192","34621,7996,3097,994,594",3,15,2024-12-05,2399,0.4,6.00,73.17
100087,narrator was absolutely perfect for such a goo...,2018-04-27,5,4.6,55145,0,Brandon Toppins,Sci fi so good,https://www.audible.com/pd/The-Hitchhikers-Gui...,"40868,9576,3215,929,557","40758,5054,1135,232,192","34621,7996,3097,994,594",4,16,2024-12-05,2414,0.4,13.90,46.78
100088,This is the kind of book that allows your imag...,2010-10-02,5,4.6,55145,0,Bill,Perfect book for a road trip,https://www.audible.com/pd/The-Hitchhikers-Gui...,"40868,9576,3215,929,557","40758,5054,1135,232,192","34621,7996,3097,994,594",6,43,2024-12-05,5178,0.4,21.85,44.75


# 파일 저장

In [24]:
df.to_csv("../EDA/audible_sampled_temp.csv")